In [87]:
from opensearch_utils import *
import os
from datasets import load_dataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.text_splitter import SpacyTextSplitter
from huggingface_utils import *
from langchain_community.vectorstores import ElasticsearchStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import OpenSearchVectorSearch
from opensearch_utils import *
import os
from datasets import load_dataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.text_splitter import SpacyTextSplitter
from huggingface_utils import *
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from elasticsearch import Elasticsearch
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [88]:
import re

def strip_non_alphanumeric(input_string):
    # Use regular expression to replace non-alphanumeric characters (excluding spaces) with an empty string
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', input_string)

def remove_quotes(input_string):
    # Remove both double and single quotes from the string
    return input_string.replace('"', '').replace("'", '')

def collapse_spaces(input_string):
    # Use regular expression to replace multiple consecutive spaces with a single space
    return re.sub(r'\s+', ' ', input_string)


In [89]:
from dotenv import load_dotenv
import os
from datasets import load_dataset

load_dotenv()  # take environment variables from .env.

HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_TOKEN')
HUGGINGFACE_USERNAME = os.getenv('HUGGINGFACE_USERNAME')
HUGGINGFACE_DATASET_NAME = os.getenv('HUGGINGFACE_DATASET_NAME')

ES_PASSWORD = os.getenv('ES_PASSWORD')

model_name = "NeuML/pubmedbert-base-embeddings"
## https://huggingface.co/NeuML/pubmedbert-base-embeddings
## %python -m spacy download en_core_web_sm

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv('API_KEY'), model_name=model_name
)

In [90]:
from elasticsearch import Elasticsearch
from getpass import getpass

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key

ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')


# client = Elasticsearch(
#   cloud_id=ELASTIC_CLOUD_ID,
#   api_key=ELASTIC_API_KEY,
  
# )

# client.info()

In [91]:

# elastic_vector_search = ElasticsearchStore(
#     es_cloud_id=ELASTIC_CLOUD_ID,
#     index_name="pubmed_index_chunks",
#     embedding=embeddings,
#     es_api_key=ELASTIC_API_KEY,
# )

# print(elastic_vector_search.client.info())
# elastic_vector_search.client.indices.create(index="pubmed_index_chunks", ignore=400)

In [92]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader

loader = HuggingFaceDatasetLoader(HUGGINGFACE_DATASET_NAME,use_auth_token=HUGGINGFACE_TOKEN,page_content_column='Abstract',cache_dir='./cache')
data = loader.load()

c:\Users\priot\anaconda3\envs\computing\Lib\site-packages\datasets\load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
c:\Users\priot\anaconda3\envs\computing\Lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [93]:
data[0]

Document(page_content='"CLINICAL CHARACTERISTICS Achondroplasia is the most common cause of disproportionate short stature Affected individuals have rhizomelic shortening of the limbs macrocephaly and characteristic facial features with frontal bossing and midface retrusion In infancy hypotonia is typical and acquisition of developmental motor milestones is often both aberrant in pattern and delayed Intelligence and life span are usually near normal although craniocervical junction compression increases the risk of death in infancy Additional complications include obstructive sleep apnea middle ear dysfunction kyphosis and spinal stenosis DIAGNOSIS TESTING Achondroplasia can be diagnosed by characteristic clinical and radiographic findings in most affected individuals In individuals in whom there is diagnostic uncertainty or who have atypical findings identification of a heterozygous pathogenic variant in FGFR3 can establish the diagnosis MANAGEMENT Treatment of manifestations Vosoriti

In [94]:
## clean page content
data = [Document(page_content= collapse_spaces(strip_non_alphanumeric(doc.page_content)).lower(),metadata=doc.metadata) for doc in data]

In [95]:
data

[Document(page_content=' clinical characteristics achondroplasia is the most common cause of disproportionate short stature affected individuals have rhizomelic shortening of the limbs macrocephaly and characteristic facial features with frontal bossing and midface retrusion in infancy hypotonia is typical and acquisition of developmental motor milestones is often both aberrant in pattern and delayed intelligence and life span are usually near normal although craniocervical junction compression increases the risk of death in infancy additional complications include obstructive sleep apnea middle ear dysfunction kyphosis and spinal stenosis diagnosis testing achondroplasia can be diagnosed by characteristic clinical and radiographic findings in most affected individuals in individuals in whom there is diagnostic uncertainty or who have atypical findings identification of a heterozygous pathogenic variant in fgfr3 can establish the diagnosis management treatment of manifestations vosorit

In [96]:
## TODO: experiment with different chunking mechanisms

text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=10,model_name=model_name,tokens_per_chunk=100)
split_data = text_splitter.split_documents(data)
len(split_data)

105373

Do hybrid search as an ensemble retriever


https://python.langchain.com/docs/integrations/vectorstores/elasticsearch

In [97]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(split_data)

result = bm25_retriever.get_relevant_documents("achondroplasia")
result

[Document(page_content='##droplasia must undergo cesarean section delivery because of small pelvic size genetic counseling achondroplasia is inherited in an autosomal dominant manner around 80 of individuals with achondroplasia have parents with average stature and have achondroplasia as the result of a de novo pathogenic variant such parents have a very low risk of having another child with achondroplasia an individual with achondroplasia who has a reproductive partner with average stature is at 50', metadata={'Journal': '1 Achondroplasia', 'Title': 'Legare JM 1', 'Authors': 'In Adam MP Feldman J Mirzaa GM Pagon RA Wallace SE Bean LJH Gripp KW Amemiya A editors GeneReviews Internet Seattle WA University of Washington Seattle 1993 2023 1998 Oct 12 updated 2023 May 11', 'Author_Information': ' 1 School of Medicine and Public Health University of Wisconsin Madison Wisconsin', 'DOI': 'Copyright 1993 2023 University of Washington Seattle GeneReviews is a registered trademark of the Univers

In [98]:
# # print(type(split_data[0]),type(split_data))
# # # embeddings.embed_documents([docs.page_content for docs in split_data])

# # for i,docs in enumerate(split_data):
# #     print("Document number:",i)
# #     print("Document metadata:",docs.metadata['Title'])
# #     embeddings.embed_documents([docs.page_content])

# invalid_rows = 0
# invalid_texts = []
# for doc in split_data:
#     try:
#         embeddings.embed_documents([doc.page_content])
#     except Exception as e:
#         print(e)
#         invalid_rows += 1
#         invalid_texts.append(doc.page_content)

# print(f"Invalid rows: {invalid_rows}")
# print(f"Invalid texts: {invalid_texts}")

In [99]:
from langchain.globals import set_debug
from langchain.globals import set_verbose

set_debug(True)
set_verbose(True)
# len(split_data[1].page_content)

## TODO: define custom index with vector field for generated questions
'''
When hybrid is enabled, the query performed will be a combination of approximate semantic search and keyword based search.

It will use rrf (Reciprocal Rank Fusion) to balance the two scores from different retrieval methods.

Every iteration of this will add data to ElasticSearch, must remove the index before
'''
## delete index first
# elastic_vector_search.client.indices.delete(index="pubmed_default", ignore=400)

## I get the same error with a different embedding model as well

db = ElasticsearchStore.from_documents(
    split_data,
    embeddings,
    es_cloud_id=ELASTIC_CLOUD_ID,
    index_name="pubmed_index_chunks",
    es_api_key=ELASTIC_API_KEY,
    distance_strategy="COSINE",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True,
    ),
)
    

db.client.indices.refresh(index="pubmed_default")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
neuro_retriever = db.as_retriever()

bm25_retriever_weight = 0.5

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, neuro_retriever], weights=[bm25_retriever_weight, 1 - bm25_retriever_weight]
)

results = ensemble_retriever.invoke("achondroplasia")
print(results)

[Document(page_content='achondroplasia is inherited in an autosomal dominant manner around 80 of individuals with achondroplasia have parents with average stature and have achondroplasia as the result of a de novo pathogenic variant such parents have a very low risk of having another child with ach', metadata={'Journal': '1 Achondroplasia', 'Title': 'Legare JM 1', 'Authors': 'In Adam MP Feldman J Mirzaa GM Pagon RA Wallace SE Bean LJH Gripp KW Amemiya A editors GeneReviews Internet Seattle WA University of Washington Seattle 1993 2023 1998 Oct 12 updated 2023 May 11', 'Author_Information': ' 1 School of Medicine and Public Health University of Wisconsin Madison Wisconsin', 'DOI': 'Copyright 1993 2023 University of Washington Seattle GeneReviews is a registered trademark of the University of Washington Seattle All rights reserved', 'Misc': 'PMID 20301331', 'Published_date': None}), Document(page_content='##oplasia when both parents have achondroplasia the risk to their offspring of havi

In [ ]:
len(results)

6

In [ ]:

query = "What is the recommended mode of delivery for pregnant women with achondroplasia?"
results = db.similarity_search(query)
print(results[0])
print([x.metadata['Published_date'] for x in results])

page_content='down from the knees or feet on playground equipment due to risk of falling onto the head or neck pregnancy management pregnant women with achondroplasia must undergo cesarean section delivery because of small pelvic size genetic counseling achondroplasia is inherited in an autosomal dominant manner' metadata={'Journal': '1 Achondroplasia', 'Title': 'Legare JM 1', 'Authors': 'In Adam MP Feldman J Mirzaa GM Pagon RA Wallace SE Bean LJH Gripp KW Amemiya A editors GeneReviews Internet Seattle WA University of Washington Seattle 1993 2023 1998 Oct 12 updated 2023 May 11', 'Author_Information': ' 1 School of Medicine and Public Health University of Wisconsin Madison Wisconsin', 'DOI': 'Copyright 1993 2023 University of Washington Seattle GeneReviews is a registered trademark of the University of Washington Seattle All rights reserved', 'Misc': 'PMID 20301331', 'Published_date': None}
[None, None, None, None]


In [ ]:
docs = db.similarity_search(
    "What is the recommended mode of delivery for pregnant women with achondroplasia?",
    filter=[{"range": {"metadata.date": {"gte": "2000-01-01"}}}],
)
print(docs[0].metadata)

IndexError: list index out of range